# train

> Perform optimization using DiffPaSS models

In [ ]:
#| default_exp train

In [ ]:
#| hide

%load_ext autoreload
%autoreload 2

In [ ]:
#| hide

from nbdev.showdoc import *

In [ ]:
#| export

# Stdlib imports
from collections.abc import Sequence
from typing import Optional, Any, Literal

# PyTorch
import torch

# DiffPaSS imports
from diffpass.base import DiffPaSSModel
from diffpass.model import (
    MatrixApply,
    PermutationConjugate,
    apply_hard_permutation_batch_to_similarity,
    TwoBodyEntropyLoss,
    MILoss,
    InterGroupSimilarityLoss,
    IntraGroupSimilarityLoss,
)

# Type aliases
IndexPair = tuple[int, int]  # Pair of indices
IndexPairsInGroup = list[IndexPair]  # Pairs of indices in a group of sequences
IndexPairsInGroups = list[IndexPairsInGroup]  # Pairs of indices in groups of sequences

## Type aliases

```python
IndexPair = tuple[int, int]  # Pair of indices
IndexPairsInGroup = list[IndexPair]  # Pairs of indices in a group of sequences
IndexPairsInGroups = list[IndexPairsInGroup]  # Pairs of indices in groups of sequences
```

In [ ]:
#| hide

# Imports for tests
import numpy as np

In [ ]:
#| export

class InformationPairing(DiffPaSSModel):
    """DiffPaSS model for information-theoretic pairing of multiple sequence alignments (MSAs)."""

    are_inputs_msas = True

    def __init__(
        self,
        # Number of sequences in each group (e.g. species) of the two MSAs
        group_sizes: Sequence[int],
        # If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc.
        fixed_pairings: Optional[IndexPairsInGroups] = None,
        # If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations
        permutation_cfg: Optional[dict[str, Any]] = None,
        # Information-theoretic measure to use. For hard permutations, these two measures are equivalent
        information_measure: Literal["MI", "TwoBodyEntropy"] = "TwoBodyEntropy",
    ):
        super().__init__()

        # Initialize permutation and matrix apply modules
        # (self.permutation and self.matrix_apply)
        self.init_permutation(
            group_sizes=group_sizes,
            fixed_pairings=fixed_pairings,
            permutation_cfg=permutation_cfg,
        )
        self.matrix_apply = MatrixApply(group_sizes=self.group_sizes)

        # Initialize information-theoretic loss module
        self.validate_information_measure(information_measure)
        self.information_measure = information_measure
        if self.information_measure == "TwoBodyEntropy":
            self.information_loss = TwoBodyEntropyLoss()
        elif self.information_measure == "MI":
            self.information_loss = MILoss()

    def forward(
        self,
        x: torch.Tensor,
        y: torch.Tensor,
    ) -> dict[str, torch.Tensor]:
        # Soft or hard permutations (list)
        perms = self.permutation()
        x_perm = self.matrix_apply(x, mats=perms)

        # Two-body entropy portion of the loss
        loss = self.information_loss(x_perm, y)

        return {"perms": perms, "x_perm": x_perm, "loss": loss}

    def prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> None:
        # Validate inputs
        self.validate_inputs(x, y, check_same_alphabet_size=True)

    def compute_losses_identity_perm(
        self, x: torch.Tensor, y: torch.Tensor
    ) -> dict[str, float]:
        # Compute hard/soft losses when using identity permutation
        self.hard_()
        with torch.no_grad():
            hard_loss_identity_perm = self.information_loss(x, y).item()
            soft_loss_identity_perm = hard_loss_identity_perm

        return {"hard": hard_loss_identity_perm, "soft": soft_loss_identity_perm}

In [ ]:
show_doc(InformationPairing)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/train.py#L36){target="_blank" style="float:right; font-size:smaller"}

### InformationPairing

>      InformationPairing (group_sizes:collections.abc.Sequence[int],
>                          fixed_pairings:Optional[list[list[tuple[int,int]]]]=N
>                          one, permutation_cfg:Optional[dict[str,Any]]=None, in
>                          formation_measure:Literal['MI','TwoBodyEntropy']='Two
>                          BodyEntropy')

*DiffPaSS model for information-theoretic pairing of multiple sequence alignments (MSAs).*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| group_sizes | Sequence |  | Number of sequences in each group (e.g. species) of the two MSAs |
| fixed_pairings | Optional | None | If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc. |
| permutation_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations |
| information_measure | Literal | TwoBodyEntropy | Information-theoretic measure to use. For hard permutations, these two measures are equivalent |

In [ ]:
def test_information_bootstrap():
    # Data: two highly correlated MSAs
    n_classes = 3
    length = 5
    size_each_group = 10
    n_groups = 10
    # Define first MSA group by group
    x_tok_by_group = [torch.randint(0, n_classes, (size_each_group, length)) for _ in range(n_groups)]
    # Within-group shuffling to control for algorithmic biases towards identity permutation
    x_tok_by_group_shuffle = [x[torch.randperm(size_each_group)] for x in x_tok_by_group]
    x_tok = torch.cat(x_tok_by_group, dim=0)
    x_tok_shuffle = torch.cat(x_tok_by_group_shuffle, dim=0)
    y_tok = (x_tok + 1) % n_classes
    x = torch.nn.functional.one_hot(x_tok).to(torch.get_default_dtype())
    x_shuffle = torch.nn.functional.one_hot(x_tok_shuffle).to(torch.get_default_dtype())
    y = torch.nn.functional.one_hot(y_tok).to(torch.get_default_dtype())

    group_sizes = [size_each_group] * n_groups

    # Model
    model = InformationPairing(group_sizes=group_sizes)
    results = model.fit_bootstrap(x_shuffle, y)
    hard_loss_identity_perm = model.compute_losses_identity_perm(x, y)["hard"]

    # Check that the hard loss of the optimized permutation is close to the ground truth
    assert np.abs(results.hard_losses[-2][-1] - hard_loss_identity_perm) < 1e-4

test_information_bootstrap()

In [ ]:
#| export

class BestHitsPairing(DiffPaSSModel):
    """DiffPaSS model for pairing of multiple sequence alignments (MSAs) by aligning their orthology networks, constructed using (reciprocal) best hits ."""

    are_inputs_msas = True

    def __init__(
        self,
        # Number of sequences in each group (e.g. species) of the two MSAs
        group_sizes: Sequence[int],
        # If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc.
        fixed_pairings: Optional[IndexPairsInGroups] = None,
        # If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations
        permutation_cfg: Optional[dict[str, Any]] = None,
        # (Smoothly extended) similarity metric to use on all pairs of aligned sequences
        similarity_kind: Literal["Hamming", "Blosum62"] = "Hamming",
        # If not ``None``, configuration dictionary containing init parameters for the internal `HammingSimilarities` or `Blosum62Similarities` object to compute similarity matrices
        similarities_cfg: Optional[dict[str, Any]] = None,
        # Whether to also compute best hits within each group (in addition to between different groups)
        compute_in_group_best_hits: bool = True,
        # If not ``None``, configuration dictionary containing init parameters for the internal `BestHits` object to compute soft/hard (reciprocal) best hits
        best_hits_cfg: Optional[dict[str, Any]] = None,
        # If not ``None``, custom callable to compute the differentiable loss between the soft/hard best hits matrices of the two MSAs
        similarities_comparison_loss: Optional[callable] = None,
        # Whether to compare the soft best hits from the MSA to permute (``x``) to the hard or soft best hits from the reference MSA (``y``)
        compare_soft_best_hits_to_hard: bool = True,
    ):
        super().__init__()

        # Initialize permutation and matrix apply modules
        # (self.permutation and self.matrix_apply)
        self.init_permutation(
            group_sizes=group_sizes,
            fixed_pairings=fixed_pairings,
            permutation_cfg=permutation_cfg,
        )
        self.matrix_apply = MatrixApply(group_sizes=self.group_sizes)

        # Validate similarity kind/config and initialize similarities module
        self.init_similarities(
            similarity_kind=similarity_kind, similarities_cfg=similarities_cfg
        )

        # Validate best hits config and initialize best hits module
        self.compute_in_group_best_hits = compute_in_group_best_hits
        self.init_best_hits(best_hits_cfg)

        self.compare_soft_best_hits_to_hard = compare_soft_best_hits_to_hard

        # Similarities comparison loss
        self.similarities_comparison_loss = similarities_comparison_loss
        if self.similarities_comparison_loss is None:
            self.effective_similarities_comparison_loss_ = InterGroupSimilarityLoss(
                group_sizes=self.group_sizes
            )
        else:
            self.effective_similarities_comparison_loss_ = (
                self.similarities_comparison_loss
            )

    def _precompute_bh(self, x: torch.Tensor, y: torch.Tensor) -> None:
        mode = self.best_hits.mode

        # Temporarily switch to hard BH
        self.best_hits.hard_()
        similarities_x = self.similarities(x)
        self.register_buffer("_bh_hard_x", self.best_hits(similarities_x))
        similarities_y = self.similarities(y)
        self.register_buffer("_bh_hard_y", self.best_hits(similarities_y))

        # Switch to soft BH
        self.best_hits.soft_()
        self.register_buffer("_bh_soft_x", self.best_hits(similarities_x))
        self.register_buffer("_bh_soft_y", self.best_hits(similarities_y))

        # Restore initial mode
        self.best_hits.mode = mode

    @property
    def _bh_y_for_soft_x(self):
        if self.compare_soft_best_hits_to_hard:
            return self._bh_hard_y
        return self._bh_soft_y

    def forward(
        self, x: torch.Tensor, y: Optional[torch.Tensor] = None
    ) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode
        assert (
            mode == self.best_hits.mode
        ), "Permutation and best hits must be either both in soft mode or both in hard mode."

        # Soft or hard permutations
        perms = self.permutation()
        x_perm = self.matrix_apply(x, mats=perms)

        # Best hits loss, with shortcut for hard permutations
        if mode == "soft":
            similarities_x = self.similarities(x_perm)
            bh_x = self.best_hits(similarities_x)
            # Ensure comparisons are soft_x-{soft,hard}_y, depending on
            # self.compare_soft_best_hits_to_hard
            loss = self.effective_similarities_comparison_loss_(
                bh_x, self._bh_y_for_soft_x
            )
        else:
            bh_x = apply_hard_permutation_batch_to_similarity(
                x=self._bh_hard_x, perms=perms
            )
            loss = self.effective_similarities_comparison_loss_(bh_x, self._bh_hard_y)

        return {
            "perms": perms,
            "x_perm": x_perm,
            "loss": loss,
        }

    def prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> None:
        # Validate inputs
        self.validate_inputs(x, y, check_same_alphabet_size=True)

        # Precompute matrices of best hits
        self._precompute_bh(x, y)

    def compute_losses_identity_perm(
        self, x: torch.Tensor, y: torch.Tensor
    ) -> dict[str, float]:
        # Precompute matrices of best hits
        self._precompute_bh(x, y)

        # Compute hard/soft losses when using identity permutation
        with torch.no_grad():
            hard_loss_identity_perm = self.effective_similarities_comparison_loss_(
                self._bh_hard_x, self._bh_hard_y
            ).item()
            soft_loss_identity_perm = self.effective_similarities_comparison_loss_(
                self._bh_soft_x, self._bh_y_for_soft_x
            ).item()

        return {"hard": hard_loss_identity_perm, "soft": soft_loss_identity_perm}

In [ ]:
show_doc(BestHitsPairing)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/train.py#L101){target="_blank" style="float:right; font-size:smaller"}

### BestHitsPairing

>      BestHitsPairing (group_sizes:collections.abc.Sequence[int],
>                       fixed_pairings:Optional[list[list[tuple[int,int]]]]=None
>                       , permutation_cfg:Optional[dict[str,Any]]=None,
>                       similarity_kind:Literal['Hamming','Blosum62']='Hamming',
>                       similarities_cfg:Optional[dict[str,Any]]=None,
>                       compute_in_group_best_hits:bool=True,
>                       best_hits_cfg:Optional[dict[str,Any]]=None,
>                       similarities_comparison_loss:Optional[<built-
>                       infunctioncallable>]=None,
>                       compare_soft_best_hits_to_hard:bool=True)

*DiffPaSS model for pairing of multiple sequence alignments (MSAs) by aligning their orthology networks, constructed using (reciprocal) best hits .*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| group_sizes | Sequence |  | Number of sequences in each group (e.g. species) of the two MSAs |
| fixed_pairings | Optional | None | If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc. |
| permutation_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations |
| similarity_kind | Literal | Hamming | (Smoothly extended) similarity metric to use on all pairs of aligned sequences |
| similarities_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `HammingSimilarities` or `Blosum62Similarities` object to compute similarity matrices |
| compute_in_group_best_hits | bool | True | Whether to also compute best hits within each group (in addition to between different groups) |
| best_hits_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `BestHits` object to compute soft/hard (reciprocal) best hits |
| similarities_comparison_loss | Optional | None | If not ``None``, custom callable to compute the differentiable loss between the soft/hard best hits matrices of the two MSAs |
| compare_soft_best_hits_to_hard | bool | True | Whether to compare the soft best hits from the MSA to permute (``x``) to the hard or soft best hits from the reference MSA (``y``) |

In [ ]:
def test_besthits_bootstrap():
    # Data: two highly correlated MSAs
    n_classes = 3
    length = 100
    size_each_group = 10
    n_groups = 10
    # Define first MSA group by group
    x_tok_by_group = [torch.randint(0, n_classes, (size_each_group, length)) for _ in range(n_groups)]
    # Within-group shuffling to control for algorithmic biases towards identity permutation
    x_tok_by_group_shuffle = [x[torch.randperm(size_each_group)] for x in x_tok_by_group]
    x_tok = torch.cat(x_tok_by_group, dim=0)
    x_tok_shuffle = torch.cat(x_tok_by_group_shuffle, dim=0)
    y_tok = (x_tok + 1) % n_classes
    x = torch.nn.functional.one_hot(x_tok).to(torch.get_default_dtype())
    x_shuffle = torch.nn.functional.one_hot(x_tok_shuffle).to(torch.get_default_dtype())
    y = torch.nn.functional.one_hot(y_tok).to(torch.get_default_dtype())

    group_sizes = [size_each_group] * n_groups

    # Model
    model = BestHitsPairing(group_sizes=group_sizes)
    results = model.fit_bootstrap(x_shuffle, y)
    target_hard_loss = model.compute_losses_identity_perm(x, y)["hard"]

    # Check that the hard loss of the optimized permutation is close to the ground truth
    assert results.hard_losses[-2][-1] / target_hard_loss > 0.7

test_besthits_bootstrap()

In [ ]:
#| export

class MirrortreePairing(DiffPaSSModel):
    """DiffPaSS model for pairing of multiple sequence alignments (MSAs) by aligning their sequence distance networks as in the Mirrortree method."""

    are_inputs_msas = True

    def __init__(
        self,
        # Number of sequences in each group (e.g. species) of the two MSAs
        group_sizes: Sequence[int],
        # If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc.
        fixed_pairings: Optional[IndexPairsInGroups] = None,
        # If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations
        permutation_cfg: Optional[dict[str, Any]] = None,
        # (Smoothly extended) similarity metric to use on all pairs of aligned sequences
        similarity_kind: Literal["Hamming", "Blosum62"] = "Hamming",
        # If not ``None``, configuration dictionary containing init parameters for the internal `HammingSimilarities` or `Blosum62Similarities` object to compute similarity matrices
        similarities_cfg: Optional[dict[str, Any]] = None,
        # If not ``None``, custom callable to compute the differentiable loss between the similarity matrix of the two MSAs. Default: `IntraGroupSimilarityLoss`
        similarities_comparison_loss: Optional[callable] = None,
    ):
        super().__init__()

        # Initialize permutation and matrix apply modules
        # (self.permutation and self.matrix_apply)
        self.init_permutation(
            group_sizes=group_sizes,
            fixed_pairings=fixed_pairings,
            permutation_cfg=permutation_cfg,
        )
        self.matrix_apply = MatrixApply(group_sizes=self.group_sizes)

        # Validate similarity kind/config and initialize similarities module
        self.init_similarities(
            similarity_kind=similarity_kind, similarities_cfg=similarities_cfg
        )

        #  Similarities comparison loss
        self.similarities_comparison_loss = similarities_comparison_loss
        if self.similarities_comparison_loss is None:
            self.effective_similarities_comparison_loss_ = IntraGroupSimilarityLoss(
                group_sizes=self.group_sizes
            )
        else:
            self.effective_similarities_comparison_loss_ = (
                self.similarities_comparison_loss
            )

    def _precompute_similarities(self, x: torch.Tensor, y: torch.Tensor) -> None:
        self.register_buffer("_similarities_hard_x", self.similarities(x))
        self.register_buffer("_similarities_hard_y", self.similarities(y))

    def forward(
        self, x: torch.Tensor, y: Optional[torch.Tensor] = None
    ) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode

        # Soft or hard permutations (list)
        perms = self.permutation()
        x_perm = self.matrix_apply(x, mats=perms)

        # Compute similarity matrix of soft- or hard-permuted x
        if mode == "soft":
            similarities_x = self.similarities(x_perm)
        else:
            similarities_x = apply_hard_permutation_batch_to_similarity(
                x=self._similarities_hard_x, perms=perms
            )

        loss = self.effective_similarities_comparison_loss_(
            similarities_x, self._similarities_hard_y
        )

        return {
            "perms": perms,
            "x_perm": x_perm,
            "loss": loss,
        }

    def prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> None:
        # Validate inputs
        self.validate_inputs(x, y, check_same_alphabet_size=True)

        # Precompute similarity matrices
        self._precompute_similarities(x, y)

    def compute_losses_identity_perm(
        self, x: torch.Tensor, y: torch.Tensor
    ) -> dict[str, float]:
        # Precompute matrices of best hits
        self._precompute_similarities(x, y)

        # Compute hard/soft losses when using identity permutation
        with torch.no_grad():
            hard_loss_identity_perm = self.effective_similarities_comparison_loss_(
                self._similarities_hard_x, self._similarities_hard_y
            ).item()
            soft_loss_identity_perm = hard_loss_identity_perm

        return {"hard": hard_loss_identity_perm, "soft": soft_loss_identity_perm}

In [ ]:
show_doc(MirrortreePairing)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/train.py#L242){target="_blank" style="float:right; font-size:smaller"}

### MirrortreePairing

>      MirrortreePairing (group_sizes:collections.abc.Sequence[int],
>                         fixed_pairings:Optional[list[list[tuple[int,int]]]]=No
>                         ne, permutation_cfg:Optional[dict[str,Any]]=None, simi
>                         larity_kind:Literal['Hamming','Blosum62']='Hamming',
>                         similarities_cfg:Optional[dict[str,Any]]=None,
>                         similarities_comparison_loss:Optional[<built-
>                         infunctioncallable>]=None)

*DiffPaSS model for pairing of multiple sequence alignments (MSAs) by aligning their sequence distance networks as in the Mirrortree method.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| group_sizes | Sequence |  | Number of sequences in each group (e.g. species) of the two MSAs |
| fixed_pairings | Optional | None | If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc. |
| permutation_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations |
| similarity_kind | Literal | Hamming | (Smoothly extended) similarity metric to use on all pairs of aligned sequences |
| similarities_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `HammingSimilarities` or `Blosum62Similarities` object to compute similarity matrices |
| similarities_comparison_loss | Optional | None | If not ``None``, custom callable to compute the differentiable loss between the similarity matrix of the two MSAs. Default: `IntraGroupSimilarityLoss` |

In [ ]:
def test_mirrortree_bootstrap():
    # Data: two highly correlated MSAs
    n_classes = 3
    length = 100
    size_each_group = 10
    n_groups = 10
    # Define first MSA group by group
    x_tok_by_group = [torch.randint(0, n_classes, (size_each_group, length)) for _ in range(n_groups)]
    # Within-group shuffling to control for algorithmic biases towards identity permutation
    x_tok_by_group_shuffle = [x[torch.randperm(size_each_group)] for x in x_tok_by_group]
    x_tok = torch.cat(x_tok_by_group, dim=0)
    x_tok_shuffle = torch.cat(x_tok_by_group_shuffle, dim=0)
    y_tok = (x_tok + 1) % n_classes
    x = torch.nn.functional.one_hot(x_tok).to(torch.get_default_dtype())
    x_shuffle = torch.nn.functional.one_hot(x_tok_shuffle).to(torch.get_default_dtype())
    y = torch.nn.functional.one_hot(y_tok).to(torch.get_default_dtype())

    group_sizes = [size_each_group] * n_groups

    # Model
    model = MirrortreePairing(group_sizes=group_sizes)
    results = model.fit_bootstrap(x_shuffle, y)
    target_hard_loss = model.compute_losses_identity_perm(x, y)["hard"]

    # Check that the hard loss of the optimized permutation is close to the ground truth
    assert results.hard_losses[-2][-1] / target_hard_loss > 0.95

test_mirrortree_bootstrap()

In [ ]:
#| export

class GraphAlignment(DiffPaSSModel):
    """DiffPaSS model for general graph alignment starting from the weighted adjacency matrices of two graphs."""

    are_inputs_msas = False

    def __init__(
        self,
        # Number of graph nodes in each group (e.g. species), assumed the same between the two graphs to align
        group_sizes: Sequence[int],
        # If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc.
        fixed_pairings: Optional[IndexPairsInGroups] = None,
        # If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations. Soft/hard permutations ``P`` act on adjacency matrices ``X`` via ``P @ X @ P.T``
        permutation_cfg: Optional[dict[str, Any]] = None,
        # If not ``None``, custom callable to compute the differentiable loss between the soft/hard-permuted adjacency matrix of graph ``x`` and the adjacency matrix of graph ``y``. Defaults to dot product between all upper triangular elements
        comparison_loss: Optional[callable] = None,
    ):
        super().__init__()

        # Initialize permutation and matrix apply modules
        # (self.permutation and self.matrix_apply)
        self.init_permutation(
            group_sizes=group_sizes,
            fixed_pairings=fixed_pairings,
            permutation_cfg=permutation_cfg,
        )
        self.permutation_conjugate = PermutationConjugate(group_sizes=self.group_sizes)

        #  Comparison loss
        self.comparison_loss = comparison_loss
        if self.comparison_loss is None:
            # Default: dot product between all upper triangular elements
            self.effective_comparison_loss_ = IntraGroupSimilarityLoss(group_sizes=None)
        else:
            self.effective_comparison_loss_ = self.comparison_loss

    def forward(self, x: torch.Tensor, y: torch.Tensor) -> dict[str, torch.Tensor]:
        mode = self.permutation.mode

        # Soft or hard permutations (list)
        perms = self.permutation()

        # Conjugate adjacency matrix x by soft/hard permutation P: P @ x @ P.T
        if mode == "soft":
            x_perm = self.permutation_conjugate(x, mats=perms)
        else:
            x_perm = apply_hard_permutation_batch_to_similarity(x=x, perms=perms)
        loss = self.effective_comparison_loss_(x_perm, y, mats=perms)

        return {
            "perms": perms,
            "x_perm": x_perm,
            "loss": loss,
        }

    def prepare_fit(self, x: torch.Tensor, y: torch.Tensor) -> None:
        # Validate inputs
        self.validate_inputs(x, y)

    def compute_losses_identity_perm(
        self, x: torch.Tensor, y: torch.Tensor
    ) -> dict[str, float]:
        # Compute hard/soft losses when using identity permutation
        with torch.no_grad():
            hard_loss_identity_perm = self.effective_comparison_loss_(
                x, y, mats=[torch.eye(s).to(x.device) for s in self.group_sizes]
            ).item()
            soft_loss_identity_perm = hard_loss_identity_perm

        return {"hard": hard_loss_identity_perm, "soft": soft_loss_identity_perm}

In [ ]:
show_doc(GraphAlignment)

---

[source](https://github.com/Bitbol-Lab/DiffPaSS/blob/main/diffpass/train.py#L343){target="_blank" style="float:right; font-size:smaller"}

### GraphAlignment

>      GraphAlignment (group_sizes:collections.abc.Sequence[int],
>                      fixed_pairings:Optional[list[list[tuple[int,int]]]]=None,
>                      permutation_cfg:Optional[dict[str,Any]]=None,
>                      comparison_loss:Optional[<built-
>                      infunctioncallable>]=None)

*DiffPaSS model for general graph alignment starting from the weighted adjacency matrices of two graphs.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| group_sizes | Sequence |  | Number of graph nodes in each group (e.g. species), assumed the same between the two graphs to align |
| fixed_pairings | Optional | None | If not ``None``, fixed pairings between groups, of the form [[(i1, j1), (i2, j2), ...], ...] where (i1, j1) are the indices of the first fixed pair in the first group to be paired, etc. |
| permutation_cfg | Optional | None | If not ``None``, configuration dictionary containing init parameters for the internal `GeneralizedPermutation` object to compute soft/hard permutations. Soft/hard permutations ``P`` act on adjacency matrices ``X`` via ``P @ X @ P.T`` |
| comparison_loss | Optional | None | If not ``None``, custom callable to compute the differentiable loss between the soft/hard-permuted adjacency matrix of graph ``x`` and the adjacency matrix of graph ``y``. Defaults to dot product between all upper triangular elements |

In [ ]:
def test_graph_alignment_bootstrap():
    # Data: two identical weighted adjacency matrices
    size_each_group = 10
    n_groups = 10
    n_samples = size_each_group * n_groups
    x = torch.exp(torch.randn((n_samples, n_samples))).to(torch.get_default_dtype())
    y = x.clone()
    # Within-group shuffling to control for algorithmic biases towards identity permutation
    rand_perm_mats = []
    for _ in range(n_groups):
        rp_mat = torch.zeros(
            (size_each_group, size_each_group), dtype=x.dtype, device=x.device
        )
        rp_mat[torch.arange(size_each_group), torch.randperm(size_each_group)] = 1
        rand_perm_mats.append(rp_mat)
    x_shuffle = apply_hard_permutation_batch_to_similarity(x=x, perms=rand_perm_mats)

    group_sizes = [size_each_group] * n_groups

    # Model
    model = GraphAlignment(group_sizes=group_sizes)
    results = model.fit_bootstrap(x_shuffle, y)
    target_hard_loss = model.compute_losses_identity_perm(x, y)["hard"]

    # Check that the hard loss of the optimized permutation is close to the ground truth
    assert results.hard_losses[-2][-1] / target_hard_loss > 0.95

test_graph_alignment_bootstrap()